In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('lab09.ok')

# Lab 9: Statistic Inference Review

Welcome to Lab 9! Over the past several weeks you've learned the fundamentals of statistic inference, from hypothesis testing to confidence intervals. This lab will review the main topics in inference. As we move on in the course you should be comfortable defining hypotheses, picking a test statistic, interpreting p-values, and building confidence intervals. 

In [1]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

# These lines load the tests.
from client.api.notebook import Notebook
ok = Notebook('lab09.ok')
_ = ok.submit()

# Overview

A hypothesis test is motivated by the data we observe or by a belief we have about the world. Many times after collecting data, we notice a trend in our data that is different from what we expect. We want to determine if that deviation can be explained by something other than random chance: is the world actually different from what we expect, or was our data just "unlucky"? In order to make an objective conclusion about the "truth", we conduct a hypothesis test.

When conducting a statistical test, we usually$^†$ follow this basic structure:
1. Define a null and alternative hypothesis.
2. Choose a p-value cutoff (usually .05 or .01).
2. Choose a test statistic and calculate the observed test statistic (from your real data).
3. Create a distribution of test statistics under the null hypothesis by simulating data.
4. Calculate a p-value using your simulated test statistics.
5. Using the p-value cutoff from step 2, determine if your data are more consistent with your null or alternative hypothesis: 
   * If the p-value is above the cutoff, the data are more consistent with the null hypothesis.
   * If the p-value is below the cutoff, the data are more consistent with the alternative hypothesis.


Depending on the type of test you want to do, each step will be a little different. You should be familiar with the different types of tests we've covered and their corresponding hypotheses and test statistics.

$^†$For some null hypotheses, we can't always follow this process. In some of those cases, we can use confidence intervals to conduct hypothesis tests. See the "Hypothesis tests using confidence intervals" section below for more information on how those work.

### Hypothesis tests using confidence intervals

Sometimes, we might not be able to simulate data under the null hypothesis. For example, if your null hypothesis is of the form "The mean of some population is 40 inches" and your alternative is of the form "The mean of the population is not 40 inches", and you observe a sample with mean 37 inches, it's not clear how you'd use the null hypothesis to sample. In a case like this, you can use a confidence interval to conduct the hypothesis test.

The basic structure is similar to the one described at the start of the lab, but with some modifications:
1. Define a null and alternative hypothesis (they must be of the form "The mean is X" and "The mean is not X").
2. Choose a p-value cutoff, and call it q.
3. Construct a (100-q)% interval using bootstrap sampling (for example, if your p-value cutoff q is .01, or 1%, then construct a 99% confidence interval).
4. Using the confidence interval, determine if your data are more consistent with your null or alternative hypothesis:
   * If the null hypothesis mean X is in your confidence interval, the data are more consistent with the null hypothesis.
   * If the null hypothesis mean X is *not* in your confidence interval, the data are more consistent with the alternative hypothesis.

Answer the following questions to test your knowledge of the different variations of hypothesis tests.

## The Data

Throughout this lab, we'll be working with data from the Gallup World Poll that is presented in the World Happiness Report, a survey of the state of global happiness. The survey ranked 155 countries by overall happiness and estimated the influence that economic production, social support, life expectancy, freedom, absence of corruption, and generosity had on population happiness. The study has been repeated for several years - we'll be looking at data from the 2016 survey.

**Run the cell below to load the dataset**

In [2]:
happiness_scores = Table.read_table("happiness_scores.csv")
happiness_scores

Participants in the study were asked to evaluate their life satisfaction from a scale from 0 (worst possible life) to 10 (best possible life). The responses for each country were averaged to create the `Happiness Score`

The columns `Economy (GDP per Capita)`, `Family`, `Health (Life Expectancy)`, `Freedom`, `Trust (Government Corruption)`, and `Generosity` estimate the extent to which each factor influences happiness, both for better or for worse. The higher the value, the more influential that factor was in calculating the country's happiness score. If you add up all the factors (in addition to a "difference from dystopia" value we excluded in the dataset), you get the happiness score.

# 1. Using TVD as a Test Statistic

Total variation distance (TVD) is a special type of test statistic that we use when we want to compare two distributions. It is often used when we observe that a set of observed proportions/probabilities is different than what we expect under the null model. 

Consider a six-sided die that we roll 6000 times. If the die is fair, we would expect that each face comes up 1/6 of the time. By random chance, a fair die won't always result in equal proportions (that is, we won't get exactly 1000 of each face). However, if we suspect that based on the data, the die might be unfair, we can conduct a hypothesis test using TVD to compare the expected [1/6, 1/6, 1/6, 1/6, 1/6, 1/6] distribution to what is actually observed.

In this part of the lab, we'll look at how we can use TVD to determine the effect that different factors have on happiness. 

Let's look at the different factors that affect happiness in the United States. Run the cell below to view the row in `us_happiness` that contains data for the United States.

In [3]:
us_happiness = happiness_scores.where("Country", "United States")
us_happiness

Remember that the columns `Economy (GDP per Capita)`, `Family`, `Health (Life Expectancy)`, `Freedom`, `Trust (Government Corruption)`, and `Generosity` describe the contribution that each factor has on the country's happiness  - the sum of these values is approximately equal to the country's Happiness Score. 

To compare the different factors, we'll look at the proportion of the happiness score that is attributed to each variable. 
You can find these proportions in the table `us_happiness_factors` after running the cell below

*Note:* the factors shown in `us_happiness` don't add up exactly to the happiness score, so we adjusted the proportions to  only account for the data we have access to. The proportions were found by dividing each Happiness Factor value by the sum of all Happiness Factor values.

In [4]:
us_happiness_factors = Table().read_table("us_happiness_factors.csv")
us_happiness_factors

**Question 1.1.** Suppose we want to test whether or not each factor contributes the same amount to the overall Happiness Score. Define the null hypothesis, alternative hypothesis, and test statistic in the cell below. Check your work with a neighbor, LA, or uGSI.

<!--
BEGIN QUESTION
name: q1_1
-->

*Write your answer here, replacing this text.*

**Question 1.2.** Write a function `calculate_tvd` that takes in the observed distribution (`obs_dist`) and expected distribution under the null hypothesis (`null_dist`) and calculates the total variation distance. Use this function to set `observed_tvd` to be equal to the observed test statistic.

<!--
BEGIN QUESTION
name: q1_2
-->

In [5]:
null_distribution = np.ones(6) * (1/6)

def calculate_tvd(obs_dist, null_dist):
    ...
    
observed_tvd = ...
observed_tvd

In [ ]:
ok.grade("q1_2");

**Question 1.3.** Create an array called `simulated_tvds` that contains 10000 simulated values under the null hypothesis. Assume that the original sample consisted of 1000 individuals. 

<!--
BEGIN QUESTION
name: q1_3
-->

In [8]:
simulated_tvds = ...

...

In [ ]:
ok.grade("q1_3");

Run the cell below to plot a histogram of your simulated test statistics, as well as the observed value of the test statistic.

In [12]:
Table().with_column("Simulated TVDs", simulated_tvds).hist()
plt.scatter(observed_tvd , 0, color='red', s=30, zorder=2);
plt.show();

**Question 1.4.** Use your simluated statistics to calculate the p-value of your test. Make sure that this number is consistent with what you observed in the histogram above.

<!--
BEGIN QUESTION
name: q1_4
-->

In [13]:
p_value_1 = ...
p_value_1

In [ ]:
ok.grade("q1_4");

**Question 1.5.** What can you conclude about how each factor contributes to the overall happiness score in the US? Explain your answer using the results of your hypothesis test. Assume a p-value cutoff of 0.05
```

BEGIN QUESTION
name: q1_5
```

*Write your answer here, replacing this text.*

# 2. A/B Testing

A/B testing allows us to determine if two numerical samples come from the same underlying distribution. We perform an A/B test if we think that knowing the "label" of an individual tells us something about its value for a numerical variable. 

In the birth weights example given in class, we suspected that knowing the smoker status of a mother gave us insight into whether their baby's birth weight would be higher or lower. If smoker status has no effect on birth weights, then the birth weights should have the same distributions for both smoker and non-smoker mothers. If mothers who are smokers have babies with lower birth weights, we would observe two different distributions for smoker and non-smoker mothers.

We can compare two distributions by finding the difference in their means. If the difference in means is small, the data suggest that the distributions are similar. If the difference in means is large, the data suggest that the distributions are different.

In this section of the lab, we'll use an A/B test to determine if happiness scores differ by geographic region.

What do the happiest countries in the world have in common? Run the cell below to view the countries with the top 5 highest happiness scores. 

In [15]:
happiness_scores.take(np.arange(5))

All five of these countries are in Western Europe! It looks like there might be an association between region and happiness score. Run the cell below to view the average happiness score for each region, sorted in descending order.

In [16]:
happiness_scores.select("Region", "Happiness Score").group("Region", np.mean).sort(1, descending = True)

There's a wide range of values between the different regions, but is this difference due to anything other than random chance? Using an A/B test, let's compare the distributions of happiness scores between Western Europe and Sub-Saharan Africa to see if there is a statistically significant difference between the two regions.

**Question 2.1.** We want to run an A/B Test to see if the happiness scores in Western Europe are higher than scores in Sub-Saharan Africa. In the cell below, define the null hypothesis, alternative hypothesis, and test statistic for your test. 

<!--
BEGIN QUESTION
name: q2_1
-->

*Write your answer here, replacing this text.*

**Question 2.2.** Create a table `we_ssa_happiness` that contains data from `happiness_scores` for the `Western Europe` and `Sub-Saharan Africa` regions. 

*Hint*: You might want to look at the different predicates that you can use with the `.where()` table method on the [Python reference page](http://data8.org/fa19/python-reference.html)

<!--
BEGIN QUESTION
name: q2_2
-->

In [17]:
regions_of_interest = make_array("Sub-Saharan Africa", "Western Europe")
we_ssa_happiness = ...
we_ssa_happiness

In [ ]:
ok.grade("q2_2");

**Question 2.3.** Define a function `calculate_test_statistic` that finds the difference between the mean happiness score in Western Europe and the mean happiness score in Sub-Saharan Africa in the table being passed through as the `tbl` argument. Use this function to set `observed_happiness_difference` to the observed test statistic in the `we_ssa_happiness` table.

<!--
BEGIN QUESTION
name: q2_3
-->

In [21]:
def calculate_test_statistic(tbl):
    ...

observed_happiness_difference = ...
observed_happiness_difference 

In [ ]:
ok.grade("q2_3");

**Question 2.4.** Define a function `shuffle_regions` that simulates one random permutation of the `we_ssa_happiness` table. The function should take no arguments and return a copy `we_ssa_happiness` where the values of `Region` have been shuffled.

<!--
BEGIN QUESTION
name: q2_4
-->

In [24]:
def shuffle_regions():
    ...

shuffle_regions()

In [ ]:
ok.grade("q2_4");

**Question 2.5.** Create an array `happiness_differences` that contains 1000 simulated values of your test statistic. 

<!--
BEGIN QUESTION
name: q2_5
-->

In [28]:
happiness_differences = ...

...

In [ ]:
ok.grade("q2_5");

Run the cell below to view a histogram of your test statistics plotted with the observed test statistic.

In [31]:
Table().with_column("Differences in Mean Happiness", happiness_differences).hist()
plt.scatter(observed_happiness_difference , 0, color='red', s=30, zorder=2);
plt.show();

**Question 2.6.** Use your simluated statistics to calculate the p-value of your test. Make sure that this number is consistent with what you observed in the histogram above.

<!--
BEGIN QUESTION
name: q2_6
-->

In [32]:
p_value_2 = ...
p_value_2

**Question 2.7.** What can you conclude the distributions of happiness score in Western Europe and Sub-Saharan Africa? Explain your answer using the results of your hypothesis test. Assume a p-value cutoff of 0.05

<!--
BEGIN QUESTION
name: q2_7
-->

*Write your answer here, replacing this text.*

# 3. Bootstrapping and Confidence Intervals

We're often interested in estimating a population parameter by calculating a statistic from a sample. However, sampling will always result in some degree of uncertainty. A sample captures a small subset of the population, so we can never be sure that a statistic (computed from only one sample) captures the complete truth. 

We can learn more about the population by going back and taking more samples. However, sampling is a very costly process and we don't always have the resources to collect more samples from the population. Instead, when we have a large representative sample of the population, we can create new samples by sampling with replacement from the original sample in a process called *bootstrapping*.  By the law of averages, the distribution of the original sample is likely to resemble the population, and the distributions of all the "resamples" are likely to resemble the original sample. Therefore, the distributions of all the resamples are likely to resemble the population as well.

<img src="bootstrap.png" width=500>

We calculate one statistic for each of our new bootstrap samples. By combining all of them, we can produce a confidence interval for the population parameter. We use this interval to estimate the parameter to some degree of confidence (typically 95%). A confidence level of 95% means that we can be 95% confident that the process resulted in a "good" interval.

In other words, before we start the process of collecting a sample of data from the population and creating a new confidence interval, there is a 95% chance that the interval will contain the population parameter. If we were to create many new confidence intervals, around 95% of those intervals will contain the population parameter. 

It's important to note that when a confidence interval has already been generated, it has either a 0% or 100% chance of containing the true population parameter. This is because the true population parameter isn't random: it's unknown, but it's fixed.

Let's use bootstrapping to see how much the top two countries differ in their happiness scores. Run the cell below to view the happiness scores and 95% confidence interval bounds for Denmark and Switzerland.

In [33]:
denmark = happiness_scores.where("Country", "Denmark")
switzerland = happiness_scores.where("Country", "Switzerland")

happiness_scores.take(np.arange(2)).select(0, 3, 4, 5)

The happiness scores are very similar, and we see that the confidence intervals overlap. Run the cell below to view the scores and confidence intervals plotted side-by-side: you don't have to understand how the plotting code works.

In [34]:
denmark_cis = make_array(denmark.column("Lower Confidence Interval").item(0),\
                         denmark.column("Upper Confidence Interval").item(0))

switzerland_cis = make_array(switzerland.column("Lower Confidence Interval").item(0),\
                         switzerland.column("Upper Confidence Interval").item(0))

plt.plot([0, 0], denmark_cis, label = "Denmark (Rank 1)", color='red')
plt.scatter(0, denmark.column("Happiness Score").item(0), color='red', s=200, zorder=2)
plt.plot([1, 1], switzerland_cis, label = "Switzerland (Rank 2)", color='blue')
plt.scatter(1, switzerland.column("Happiness Score").item(0), color='blue', s=200, zorder=2)
plt.legend(loc = "right")
plt.xlim(-1,4.5)
plt.ylabel("Happiness Score")
plt.title("Top Two Happiness Scores")
plt.tick_params(
    axis='x',          
    which='both',      
    bottom=False,      
    top=False,         
    labelbottom=False) 

Visually, the intervals look very similar, but we can't definitively say that there's no difference in their happiness scores without running a hypothesis test. If we decrease the width of the confidence interval, we can get a tighter bound on what the mean happiness score in Denmark could be. 

**Question 3.1.** How can we reduce the width of a 95% confidence interval of sample means?

<!--
BEGIN QUESTION
name: q3_1
-->

*Write your answer here, replacing this text.*

To see if Denmark actually has a higher happiness score than Switzerland, pretend we went out and created a random sample that contains more Danish residents than the original study. We recorded their responses in the table `denmark_scores`. Run the cell below to see data from our new sample.

In [35]:
denmark_scores = Table.read_table("denmark_scores.csv")
print("Happiness Score (Mean Happiness Level) of Sample:", np.mean(denmark_scores.column(0)))
print("Sample Size:", denmark_scores.num_rows)
denmark_scores

In [36]:
denmark_scores.hist(0, bins=np.arange(-.5, 10.5, 1))

Using this new sample, we'll create new bootstrap samples to produce a 95% confidence interval of Denmark's mean happiness scores. We can use this interval to test the following hypotheses at a 0.05 significant level: 

> **Null Hypothesis:** The mean happiness score for Denmark and Switzerland is the same. Any difference in happiness score is due to random chance. 

> **Alternative Hypothesis:** The mean happiness score for Denmark is different than the mean happiness score for Switzerland.

If the confidence interval of Denmark's mean happiness scores contains Switzerland's mean happiness score (7.509), the data are more consistent with the null hypothesis. If the confidence interval of Denmark's mean happiness scores does not contain Switzerland's mean happiness score (7.509), the data are more consistent with the alternative hypothesis.

**Question 3.2.** Define a function `bootstrap_sample` that generates a bootstrap sample from `tbl`, a one column table that looks like `denmark_scores`. The function should return the mean happiness score from the bootstrap sample.

Call your function once on `denmark_scores` to make sure it works.

<!--
BEGIN QUESTION
name: q3_2
-->

In [37]:
def bootstrap_sample(tbl):
    ...

bootstrap_sample(denmark_scores)

In [ ]:
ok.grade("q3_2");

**Question 3.3.** Create an array `denmark_means` that contains the mean happiness scores from 1000 different bootstrap samples.

<!--
BEGIN QUESTION
name: q3_3
-->

In [40]:
repetitions = ...

denmark_means = ...

...

In [ ]:
ok.grade("q3_3");

**Question 3.4.** Create a 95% confidence interval of mean happiness scores. Assign `bootstrap_ci_left_end` to the lower bound of the interval and `bootstrap_ci_right_end` to the upper bound of the interval.

<!--
BEGIN QUESTION
name: q3_4
-->

In [43]:
bootstrap_ci_left_end = ...
bootstrap_ci_right_end = ...
bootstrap_ci_left_end, bootstrap_ci_right_end

In [ ]:
ok.grade("q3_4");

Now suppose that we didn't have access to Python and can't create any bootstrap samples. However, since the happiness score is the sample mean of our happiness levels, we know that they are normally distributed under the Central Limit Theorem. We also know that in a normal distribution, approximately 95% of values are contained within 2 standard deviations of the mean. Using this information, we can still construct a 95% confidence interval with only the sample mean and standard deviation using this formula:

$$\text{Lower bound of 95% CI} = \text{Sample Mean} - 2 * \text{SD of Sample Means}$$
$$\text{Upper bound of 95% CI} = \text{Sample Mean} + 2 * \text{SD of Sample Means}$$

Where the formula for the standard deviation of sample means is given as the following: 

$$\text{SD of Sample Means} = \frac{\text{SD of Population}}{\sqrt{\text{Sample Size}}}$$

When we have a large random sample that is representative of our population, the standard deviation of the sample gives a good estimate for the standard deviation of the population. Therefore, we can use the following formula to approximate the standard deviation of sample means if we don't know the population standard deviation.

$$\text{SD of Sample Means} \approx \frac{\text{SD of Sample}}{\sqrt{\text{Sample Size}}}$$

**Question 3.5.** Create a 95% confidence interval of the happiness score (mean happiness level) for Denmark. You may not use any form of resampling or simulation to find your interval bounds. Your confidence interval should be very similar to the one you found in Question 3.4. 

<!--
BEGIN QUESTION
name: q3_5
-->

In [47]:
sample_mean = np.mean(denmark_scores.column(0)) 
sample_sd = np.std(denmark_scores.column(0))
sd_of_sample_mean = ...
left_end = ...
right_end = ...
left_end, right_end

In [ ]:
ok.grade("q3_5");

Run the cell below to plot the simulated mean happiness scores for Denmark and the observed mean happiness score for Switzerland:
* The original confidence interval for Denmark's happiness scores from `happiness_scores` is plotted as a pink line.
* The confidence interval created by our bootstrap procedure is plotted as a yellow line.
* The happiness score for Switzerland is plotted as a red dot.

In [51]:
Table().with_column("Simulated Mean Happiness Scores", denmark_means).hist()
plt.plot(make_array(denmark.column("Lower Confidence Interval").item(0),\
                    denmark.column("Upper Confidence Interval").item(0)), \
         make_array(0, 0), color='pink', lw=7, zorder=1)
plt.plot(make_array(left_end, right_end), make_array(0, 0), color='yellow', lw=7, zorder=1)
plt.scatter(switzerland.column("Happiness Score").item(0), 0, color='red', s=200, zorder=2);

**Question 3.6.** Interpret the results of the hypothesis test by assigning `happiness_ci_statements` to an array of integer(s) that correspond to the true statement(s).

1. The data are more consistent with the alternative hypothesis, since our confidence interval is contained inside the original study's confidence interval.

2. The data are more consistent with the null hypothesis, since our confidence interval contains Switzerland's happiness score.

3. The happiness levels of individual Danish residents in our sample are normally distributed.

4. The mean happiness levels (happiness scores) of our bootstrapped samples are normally distributed

<!--
BEGIN QUESTION
name: q3_6
-->

In [52]:
happiness_ci_statements = make_array(2, 4)

In [ ]:
ok.grade("q3_6");

Congratulations! You've completed Lab 9, wrapping up the statistical inference portion of the course. 

Be sure to 
- **run all the tests** (the next cell has a shortcut for that), 
- **Save and Checkpoint** from the `File` menu,
- **run the last cell to submit your work**,
- and ask one of the staff members to check you off.

In [ ]:
# For your convenience, you can run this cell to run all the tests at once!
import os
print("Running all tests...")
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]
print("Finished running all tests.")

In [ ]:
# Run this cell to submit your work *after* you have passed all of the test cells.
# It's ok to run this cell multiple times. Only your final submission will be scored.
_ = ok.submit()